In [1]:
import pandas as pd
import torch.utils.data
from datasets import AppliancePredictionDataset

ModuleNotFoundError: No module named 'torch'

In [4]:
raw_ds = pd.read_csv('./data/Database_ready_v2.csv', low_memory=False)
y = raw_ds["Presence_phone_charger"].fillna(0).values
ds = AppliancePredictionDataset("./data/numeric_data.csv", y)

NameError: name 'AppliancePredictionDataset' is not defined

In [ ]:
raw_ds['Presence_phone_charger'].fillna(0)

## Model Creation

In [ ]:
from torch import nn
import torch.optim as optim
from tqdm import tqdm


In [ ]:
batch_size = 128
EPOCHS = 1000
learning_rate = 1e-3
CONFIDENCE_THRESHOLD = 0.5

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(ds, batch_size=batch_size, shuffle=True)

for i, j in train_dataloader:
    print(i.shape, j.shape)
    break

In [ ]:
model = nn.Sequential(
    nn.Linear(49, 16),
    # nn.Linear(64, 32),
    # nn.Linear(32, 32),
    # nn.Linear(32, 32),
    # nn.Linear(32, 32),
    # nn.Linear(32, 16),
    nn.Linear(16, 1),
    nn.Sigmoid())

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [5]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('./logs')

ModuleNotFoundError: No module named 'torch'

In [6]:
from sklearn.metrics import accuracy_score


def calculate_accuracy(model: nn.Module, dataloader: torch.utils.data.DataLoader):
    y_pred = []
    y_true = []
    model.eval()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X).squeeze(1)
        y_pred += [( 1 if(i > CONFIDENCE_THRESHOLD) else 0 )  for i in pred ]
        y_true += list(y.tolist())

    return accuracy_score(y_true=y_true, y_pred=y_pred)



NameError: name 'nn' is not defined

In [ ]:
from torch.utils.tensorboard import SummaryWriter

def train_loop(dataloader: torch.utils.data.DataLoader, model: nn.Module, loss_fn,
               optimizer: optim.Optimizer, writer: SummaryWriter):
    # size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X).squeeze(1)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        writer.add_scalar('./logs/Loss/train', loss.item(), batch)

        if batch % 500 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}")
            



In [ ]:
for epoch in range(EPOCHS):
    train_loop(dataloader=train_dataloader, model=model, loss_fn=criterion, optimizer=optimizer, writer=writer)
    acc = calculate_accuracy(model, train_dataloader)
    writer.add_scalar('./logs/Accuracy/train', acc, epoch) 
    print(f"Accuracy: {acc*100:.2f}%")
    # torch.save(model.state_dict(), f'./checkpoints/model_{epoch}.pt')
    # torch.save(optimizer.state_dict(), f'./checkpoints/optimizer_{epoch}.pt')

writer.flush()

In [ ]:
print(calculate_accuracy(model=model, dataloader=train_dataloader))